In [1]:
import requests

url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"
data = requests.get(url)

In [2]:
from bs4 import BeautifulSoup as bsp
import time

soup = bsp(data.text)
time.sleep(5)

table = soup.select("table.stats_table")[0]

In [3]:
links = table.find_all("a")
links = [l.get("href") for l in links]
links = [l for l in links if "/squads/" in l]
team_urls = [f"https://fbref.com{l}" for l in links]
team_url = team_urls[0]
data = requests.get(team_url)

In [4]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

C:\Users\sethr\AppData\Local\Temp\ipykernel_39612\4209044294.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [5]:
soup = bsp(data.text)
time.sleep(5)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [6]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.columns = shooting.columns.droplevel()

C:\Users\sethr\AppData\Local\Temp\ipykernel_39612\1021631249.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [7]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [8]:
url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"
all_matches = []
for i in range(2024, 2019, -1):
    data = requests.get(url)
    soup = bsp(data.text)
    table = soup.select("table.stats_table")[0]

    links = table.find_all("a")
    links = [l.get("href") for l in links]
    links = [l for l in links if "/squads/" in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous = soup.select("a.prev")[0].get("href")
    url = f"https://fbref.com{previous}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = bsp(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = i
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(5)

C:\Users\sethr\AppData\Local\Temp\ipykernel_39612\1716375929.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\sethr\AppData\Local\Temp\ipykernel_39612\1716375929.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\sethr\AppData\Local\Temp\ipykernel_39612\1716375929.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\sethr\AppData\Local\Temp\ipykernel_39612\1716375929.py:24: FutureWarning: Passing literal 

In [9]:
df = pd.concat(all_matches)

df.to_csv("matches.csv")
